# Estimating Regional Household Visitation

This notebook accompanies the paper *'Household visitation during the COVID-19 pandemic' by 
Stuart Ross, George Breckenridge, Mengdie Zhuang, and Ed Manley*, published in Scientific Reports. 

Code segments have been extracted from the Cuebiq workbench platform, and as such some methods have been broken up and simplified to exclude database queries previously integrated within the code. 

The notebook contains the methods used to: 

 1. Estimation of visitations
 2. Estimation of 'home and work' activity
 3. Removing events near Greenspace and POIs, and far from residential buildings
 5. Counting weekly active users
 6. Household visitation rate H<sub>l,t</sub>
 7. Estimating impact of events  
 

## Libraries

In [ ]:
import warnings
warnings.filterwarnings("ignore")

import pandas as pd
import geopandas as gpd
import numpy as np
from sklearn.cluster import DBSCAN
import datetime as dt
from linearmodels import PanelOLS


## Constants and model parameters

These current settings are in line with those described in the paper.


In [ ]:
kms_per_radian = 6371.0088
epsilon = 0.030 / kms_per_radian
min_samples = 3
POI_buffer=30
residential_buffer=50
baseline_date=dt.date(2020,3,2)

## 1.   Estimation of visitations

In [ ]:
'''
Dataframe trace_day contains all GPS traces captured in one day - ["user_id", "lat", "lon", "event_timestamp"]
'''

def estimate_visitations(trace_day, epsilon = epsilon, min_samples=min_samples):
    
    cluster_list=[]

    # get visiation clusters of each user in one day using DBSCAN                  
    for user_id, trace_id in trace_day.groupby('user_id'):
        
        cluster = DBSCAN(
            eps=epsilon, 
            min_samples=min_samples, 
            algorithm='ball_tree', 
            metric='haversine'
        ).fit(np.radians(trace_id[['lat','lon']].values))   
        
        if all(item == -1 for item in cluster.labels_): 
            continue
        else:
            # compute daily stopping threshold for each user
            time_twostd = max(2*trace_id['event_timestamp'].diff().std(),1800)     
            
            trace_id['cluster_id'] = cluster.labels_

            for cluster_id, cluster_group in trace_id.groupby('cluster_id'): 
                    if cluster_id == -1: 
                        continue

                    # compute number of visits for each cluster                   
                    timestamp_array=cluster_group["event_timestamp"].values
                    
                    time_diff = np.diff(timestamp_array) 
                    
                    index_clusterbreak=np.append(np.where(time_diff>time_twostd)[0],len(timestamp_array)-1)
                    
                    counter=len(index_clusterbreak)

                    for index,item in enumerate(index_clusterbreak):
                        if index == 0: 
                            diff= timestamp_array[item]-timestamp_array[0]
                        else: 
                            diff=timestamp_array[item]-timestamp_array[index_clusterbreak[index-1]+1]
                            
                        if diff < 900: 
                            counter =counter - 1

                    cluster_list.append([user_id, cluster_id, cluster_group['lat'].mean(),cluster_group['lon'].mean(),counter])  

    return cluster_list


## 2. Estimation of 'home and work' activity 

In [ ]:
'''
Dataframe visitation_twoweeks contains all visits identified in a two-weeks period - 
["user_id","lat", "lon", "num_of_visits","date"]
'''

def estimate_homework(visitation_twoweeks, epsilon = epsilon, min_samples=min_samples):
    
    homework_list=[]

    #return clusters of visitations performed by one user in two weeks using DBSCAN                  
    for user_id, trace_id in visitation_twoweeks.groupby('user_id'):         
        
        cluster = DBSCAN(
            eps=epsilon, 
            min_samples=min_samples, 
            algorithm='ball_tree', 
            metric='haversine'
        ).fit(np.radians(trace_id[['lat','lon']].values))      
        
        trace_id['cluster_id'] = cluster.labels_

        if all(item == -1 for item in cluster.labels_): 
            continue
            
        else:  
            
            # return two most visited locations, with the number of days visited 
            # these locations and the number of visits
            candidate_list=[]
            
            for cluster_id, cluster_group in trace_id.groupby('cluster_id'):
                
                if cluster_id == -1: 
                    continue 
                    
                candidate_list.append([user_id, cluster_group['lat'].mean(), cluster_group['lon'].mean(),
                                       len(cluster_group.date.unique()),sum(cluster_group.num_of_visits)]) 

            homework_list.extend([candidate_list[i] for i in np.array(candidate_list)[:,-1].argsort()[-2:]])
            
    return homework_list


## 3. Removing events near Greenspace and POIs, and far from residential buildings

In [ ]:
'''
Dataframe POI is extracted from the Ordnance Survey POI layer -  ["id","lat", "lon","cat_num"]
Dataframe visit contains ["user_id","lat", "lon", "num_of_visits","date"]
GeoDataframe greenspaces is extracted from the Ordnance Survey -  ["id","lat", "lon","geometry"]
Dataframe residential is extracted from the Ordnance Survey AddressBase dataset -  ["id","lat", "lon"]
All gemetory are projected using epsg:27700
'''

visit_gpd = gpd.GeoDataFrame(visit, geometry=gpd.points_from_xy(visit.lon, visit.lat))

# remove points in green space
visit_gpd = gpd.sjoin(visit_gpd, greenspaces, how='left', op='within')
visit_gpd = visit_gpd[visit_gpd['id'].isnull()]

# remove points closer to POIs
POI = gpd.GeoDataFrame(POI, geometry=gpd.points_from_xy(POI.lon, POI.lat))
POI['geometry'] = POI.geometry.buffer(POI_buffer)
temp_within = gpd.sjoin(visit_gpd, POI, op='within')
visit_gpd = visit_gpd[~visit_gpd.index.isin(temp_within.index)]

# remove if too far from residential buildings
residential = gpd.GeoDataFrame(residential, geometry=gpd.points_from_xy(residential.lon, residential.lat))
residential['geometry'] = residential.geometry.buffer(residential_buffer)
visit_gpd = gpd.sjoin(visit_gpd, residential, op='within')


## 4. Counting weekly active users

In [ ]:
'''
Dataframe homework contains all home and work locations identified using two weeks raw data - ["user_id","lat", "lon", "num_of_days","num_of_visits"]
This homework datafame is updated every month
Dataframe active_users contains all users appeared in a given day for the current month- ["user_id","date"]
'''

def filter_active_users(active_users,homework):
    
    user_list = list(homework['user_id'].unique())
    
    active_users = active_users[active_users['cuebiq_id'].isin(user_list)]
    
    return active_users


## 5. Household Visitation Rate H<sub>l,t</sub>

In [ ]:
'''
Dataframe house_visit contains - ["date","LAD20NM", "total_users", "made_visit","percent"]
'''

baseline=house_visit[house_visit.date < pd.to_datetime(baseline_date)].groupby(["LAD20NM","dayofweek"]).percent.mean().reset_index()

baseline.rename(columns={"percent":"baseline"},inplace=True)

house_visit=house_visit.merge(baseline, how="left",on=["LAD20NM","dayofweek"])

house_visit["week"]=hv_rate.date.dt.isocalendar().week

house_visit["h_rate"]=(house_visit.percent-house_visit.baseline)/house_visit.baseline

house_visit.sort_values(by=["LAD20NM",'date'],inplace=True)

house_visit.h_rate=house_visit.h_rate.fillna(0)

house_visit["SMA_7"]=house_visit.groupby("LAD20NM").h_rate.transform(lambda x: x.rolling(7, 7,center=True).mean())


## 6. Estimating Impact of Events  

In [ ]:
def cutoff(date,threhold): # threshold
    if isinstance(threhold, str): 
        temp=(date- dt.datetime.strptime(threhold,'%Y-%m-%d')).dt.days
    else: 
        temp=(date- threhold).dt.days
    return temp

def estimate_effect(house_visit, areas, date_left,threshold,date_right):
    '''
    dataframe house_visit is resulted from 5 (above)
    areas refer to a list of local authority
    threhold, is the date when the poliy taken effect
    date_left and data_right mark the 15 days window
    '''
    
    temp=merged[(house_visit.date<date_right)&(date_left<house_visit.date)&(house_visit.LAD20NM.isin(areas))].copy()
    
    temp.loc[:,"day_norm"]=cutoff(temp['date'],threshold)
    
    temp = temp.assign(threshold=(temp["day_norm"] >= 0).astype(int))
    
    temp = temp.set_index(['LAD20NM','date'])
        
    model = PanelOLS.from_formula(formula='h_rate ~ 1+threshold+ day_norm+threshold*day_norm + EntityEffects', data=temp)
    
    res = model.fit(cov_type='clustered', cluster_entity=True,cluster_time=True)
    
    return res